In [2]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import toolkitEngineering
import importlib
importlib.reload(toolkitEngineering)

<module 'toolkitEngineering' from 'd:\\Projetos\\ETL-ML-Dengue\\src\\engineering\\toolkitEngineering.py'>

In [3]:
## Estabelecendo diretórios base
diretorio_src_etl = os.getcwd()
diretorio_src = Path(diretorio_src_etl).parent.absolute()
diretorio_base = Path(diretorio_src).parent.absolute()
diretorio_data = os.path.join(diretorio_base, 'data')

# Bronze
diretorio_bronze = os.path.join(diretorio_data, 'bronze')
diretorio_sinan_bronze = os.path.join(diretorio_bronze, 'sinan')
diretorio_sinan_bronze_csv = os.path.join(diretorio_sinan_bronze, 'csv')

# Silver
diretorio_silver = os.path.join(diretorio_data, 'silver')
diretorio_sinan_silver = os.path.join(diretorio_silver, 'sinan')
diretorio_sinan_silver_tsv = os.path.join(diretorio_sinan_silver, 'tsv')

## Criação de pastas para armazenar os arquivos
os.makedirs(os.path.join(diretorio_data), exist_ok=True)
os.makedirs(os.path.join(diretorio_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_sinan_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_sinan_silver_tsv), exist_ok=True)




In [4]:
arquivos_csv = [arquivo for arquivo in os.listdir(diretorio_sinan_bronze_csv) if arquivo.endswith("csv")]
arquivos_csv = arquivos_csv[-5:]
arquivos_tsv = [arquivo.rstrip("csv") + "tsv" for arquivo in arquivos_csv]
arquivos_tsv

['head_10000_DENGBR10.tsv',
 'head_10000_DENGBR11.tsv',
 'head_10000_DENGBR12.tsv',
 'head_10000_DENGBR13.tsv',
 'head_10000_DENGBR14.tsv']

In [5]:
# Função para converter NU_IDADE_N para idade em anos
def convert_to_years(age):
    if pd.isnull(age):
        return np.nan
    if len(str(age)) < 4:
        return np.nan
    age_str = str(age)
    prefix = age_str[0]
    value = int(age_str[1:])
    
    if prefix == '1':  # Hora
        return value / (24 * 365)  # Aproximação para converter horas em anos
    elif prefix == '2':  # Dia
        return value / 365  # Aproximação para converter dias em anos
    elif prefix == '3':  # Mês
        return value / 12  # Aproximação para converter meses em anos
    elif prefix == '4':  # Ano
        return value
    else:
        return np.nan




In [8]:
colunas_decodificar = {
    "CLASSI_FIN": {
        5: "Descartado",
        10: "Dengue",
        11: "Dengue com sinais de alarme",
        12: "Dengue grave",
        13: "Chikungunya",
        1: "Dengue",
        2: "Dengue com sinais de alarme",
        3: "Dengue grave",
        4: "Dengue grave"
    },
    "COMPLICA": {
        1: "Alterações neurológicas",
        2: "Disfunção cardiorespiratória",
        3: "Insuficiência hepática",
        4: "Plaquetas < 100",
        8: "Não se enquadra nos critérios de FHD"
    },
    "CON_FHD": {
        1: "Grau I",
        2: "Grau II",
        3: "Grau III",
        4: "Grau IV"
    },
    "CRITERIO": {
        1: "Laboratório",
        2: "Clínico Epidemiológico",
        3: "Em investigação"
    },
    "CS_ESCOL_N": {
        43: "Analfabeto",
        1: "1ª a 4ª série incompleta do EF",
        2: "4ª série completa do EF (antigo 1° grau)",
        3: "5ª à 8ª série incompleta do EF (antigo ginásio ou 1° grau)",
        4: "Ensino fundamental completo (antigo ginásio ou 1° grau)",
        5: "Ensino médio incompleto (antigo colegial ou 2° grau)",
        6: "Ensino médio completo (antigo colegial ou 2° grau)",
        7: "Educação superior incompleta",
        8: "Educação superior completa",
        9: "Ignorado",
        10: "Não se aplica"
    },
    "CS_GESTANT": {
        1: "1º Trimestre",
        2: "2º Trimestre",
        3: "3º Trimestre",
        4: "Idade gestacional ignorada",
        5: "Não",
        6: "Não se aplica",
        9: "Ignorado"
    },
    "CS_RACA": {
        1: "branca",
        2: "preta",
        3: "amarela",
        4: "parda",
        5: "indígena",
        9: "Ignorado"
    },
    "CS_SEXO": {
        "M": "Masculino",
        "F": "Feminino",
        "I": "Ignorado"
    },
    "EPISTAXE": {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    "EVIDENCIA": {
        1: "Hemoconcentração",
        2: "Derrames cavitários",
        3: "Hipoproteinemia"
    },
    "EVOLUCAO": {
        1: "cura",
        2: "óbito por dengue",
        3: "óbito por outras causas",
        4: "óbito em investigação",
        9: "ignorado"
    },
    "GENGIVO": {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    "HEMATURA": {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    "HISTOPA_N": {
        1: "Positivo",
        2: "Negativo",
        3: "Inconclusivo",
        4: "Não realizado"
    },
    "HOSPITALIZ": {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    "IMUNOH_N": {
        1: "Positivo",
        2: "Negativo",
        3: "Inconclusivo",
        4: "Não realizado"
    },
    "LACO_N": {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    "MANI_HEMOR": {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    "METRO": {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    "PETEQUIAS": {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    "PLASMATICO": {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    "RESUL_NS1": {
        1: "Positivo",
        2: "Negativo",
        3: "Inconclusivo",
        4: "Não realizado"
    },
    "RESUL_PCR_": {
        1: "Positivo",
        2: "Negativo",
        3: "Inconclusivo",
        4: "Não realizado"
    },
    "RESUL_SORO": {
        1: "Reagente",
        2: "Não Reagente",
        3: "Inconclusivo",
        4: "Não realizado"
    },
    "RESUL_VI_N": {
        1: "Positivo",
        2: "Negativo",
        3: "Inconclusivo",
        4: "Não realizado"
    },
    "SANGRAM": {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    "SOROTIPO": {
        1: "DEN 1",
        2: "DEN 2",
        3: "DEN 3",
        4: "DEN 4"
    },
    "TPAUTOCTO": {
        1: "Sim",
        2: "Não",
        3: "Indeterminado"
    },
    "ACIDO_PEPT": {
        1: "Sim",
        2: "Não"
    },
    "ALRM_ABDOM": {
        1: "Sim",
        2: "Não"
    },
    "ALRM_HEMAT": {
        1: "Sim",
        2: "Não"
    },
    "ALRM_HEPAT": {
        1: "Sim",
        2: "Não"
    },
    "ALRM_HIPOT": {
        1: "Sim",
        2: "Não"
    },
    "ALRM_LETAR": {
        1: "Sim",
        2: "Não"
    },
    "ALRM_LIQ": {
        1: "Sim",
        2: "Não"
    },
    "ALRM_PLAQ": {
        1: "Sim",
        2: "Não"
    },
    "ALRM_SANG": {
        1: "Sim",
        2: "Não"
    },
    "ALRM_VOM": {
        1: "Sim",
        2: "Não"
    },
    "ARTRALGIA": {
        1: "Sim",
        2: "Não"
    },
    "ARTRITE": {
        1: "Sim",
        2: "Não"
    },
    "AUTO_IMUNE": {
        1: "Sim",
        2: "Não"
    },
    "CEFALEIA": {
        1: "Sim",
        2: "Não"
    },
    "CONJUNTVIT": {
        1: "Sim",
        2: "Não"
    },
    "DIABETES": {
        1: "Sim",
        2: "Não"
    },
    "DOR_COSTAS": {
        1: "Sim",
        2: "Não"
    },
    "DOR_RETRO": {
        1: "Sim",
        2: "Não"
    },
    "EXANTEMA": {
        1: "Sim",
        2: "Não"
    },
    "FEBRE": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_AST": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_CONSC": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_CONV": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_ENCH": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_EXTRE": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_HEMAT": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_HIPOT": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_INSUF": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_MELEN": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_METRO": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_MIOC": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_ORGAO": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_PULSO": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_SANG": {
        1: "Sim",
        2: "Não"
    },
    "GRAV_TAQUI": {
        1: "Sim",
        2: "Não"
    },
    "HEMATOLOG": {
        1: "Sim",
        2: "Não"
    },
    "HEPATOPAT": {
        1: "Sim",
        2: "Não"
    },
    "HIPERTENSA": {
        1: "Sim",
        2: "Não"
    },
    "LACO": {
        1: "Sim",
        2: "Não"
    },
    "LEUCOPENIA": {
        1: "Sim",
        2: "Não"
    },
    "MIALGIA": {
        1: "Sim",
        2: "Não"
    },
    "NAUSEA": {
        1: "Sim",
        2: "Não"
    },
    "PETEQUIA_N": {
        1: "Sim",
        2: "Não"
    },
    "RENAL": {
        1: "Sim",
        2: "Não"
    },
    "RESUL_PRNT": {
        1: "Reagente",
        2: "Não Reagente",
        3: "Inconclusivo",
        4: "Não realizado"
    },
    "RES_CHIKS1": {
        1: "Reagente",
        2: "Não Reagente",
        3: "Inconclusivo",
        4: "Não realizado"
    },
    "RES_CHIKS2": {
        1: "Reagente",
        2: "Não Reagente",
        3: "Inconclusivo",
        4: "Não realizado"
    },
    "VOMITO": {
        1: "Sim",
        2: "Não"
    }
}


colunas_importantes = [
    "CLASSI_FIN", "COMPLICA", "COMUNINF", "CON_FHD", "COPAISINF", "COUFINF", "CRITERIO", "CS_ESCOL_N", "CS_GESTANT", "CS_RACA",
    "CS_SEXO", "DT_INTERNA", "DT_INVEST", "DT_NOTIFIC", "DT_NS1", "DT_OBITO", "DT_PCR", "DT_SIN_PRI", "DT_SORO", "DT_VIRAL", "EPISTAXE",
    "EVIDENCIA", "EVOLUCAO", "GENGIVO", "HEMATURA", "HISTOPA_N", "HOSPITALIZ", "ID_MUNICIP", "ID_PAIS", "ID_UNIDADE", "IMUNOH_N",
    "LACO_N", "MANI_HEMOR", "METRO", "MUNICIPIO", "NU_ANO", "NU_IDADE_N", "PETEQUIAS", "PLAQ_MENOR", "PLASMATICO", "RESUL_NS1", "RESUL_PCR_",
    "RESUL_SORO", "RESUL_VI_N", "SANGRAM", "SEM_NOT", "SEM_PRI", "SOROTIPO", "TPAUTOCTO", "UF", "ACIDO_PEPT", "ALRM_ABDOM", "ALRM_HEMAT",
    "ALRM_HEPAT", "ALRM_HIPOT", "ALRM_LETAR", "ALRM_LIQ", "ALRM_PLAQ", "ALRM_SANG", "ALRM_VOM", "ARTRALGIA", "ARTRITE",
    "AUTO_IMUNE", "CEFALEIA", "CONJUNTVIT", "DIABETES", "DOR_COSTAS", "DOR_RETRO", "DT_ALRM", "DT_CHIK_S1", "DT_CHIK_S2", "DT_GRAV",
    "DT_NASC", "DT_PRNT", "EXANTEMA", "FEBRE", "GRAV_AST", "GRAV_CONSC", "GRAV_CONV", "GRAV_ENCH", "GRAV_EXTRE", "GRAV_HEMAT", "GRAV_HIPOT",
    "GRAV_INSUF", "GRAV_MELEN", "GRAV_METRO", "GRAV_MIOC", "GRAV_ORGAO", "GRAV_PULSO", "GRAV_SANG", "GRAV_TAQUI", "HEMATOLOG", "HEPATOPAT",
    "HIPERTENSA", "LACO", "LEUCOPENIA", "MIALGIA", "NAUSEA", "PETEQUIA_N", "RENAL", "RESUL_PRNT", "RES_CHIKS1", "RES_CHIKS2", "VOMITO"
]

column_map = {
    "DT_INTER": "DT_INTERNA"
}

    # Lista das colunas que queremos tentar converter para inteiros
columns_to_convert = [
    "comuninf", "con_fhd", "copaisinf", "coufinf",
    "id_municip", "id_pais", "id_unidade",
    "nu_ano", "nu_idade_n",
    "sem_not", "sem_pri", "nu_mes", "uf"
]

In [9]:
for arquivo_csv, arquivo_tsv in zip(arquivos_csv, arquivos_tsv):
    df = pd.read_csv(os.path.join(diretorio_sinan_bronze_csv, arquivo_csv), sep=',')

    for coluna in colunas_importantes:
        if coluna not in df.columns:
            df[coluna] = np.nan
            continue
    df = df[colunas_importantes]
    for coluna in colunas_decodificar:
        df[coluna] = df[coluna].apply(lambda x: toolkitEngineering.aplicar_decoficador(x, coluna, colunas_decodificar))
    df.replace("NA", np.nan, inplace=True)
    df.replace("", np.nan, inplace=True)
    df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
    df['IDADE_ANOS'] = df['NU_IDADE_N'].apply(convert_to_years)
    df['NU_MES'] = df['DT_NOTIFIC'].apply(lambda x: x.split('-')[1].lstrip("0"))
    df = df.rename(columns=column_map)
    
    df.columns = df.columns.str.lower()
    
    for column in columns_to_convert:
        df[column] = pd.to_numeric(df[column], errors='coerce').astype('Int64')

    df.to_csv(os.path.join(diretorio_sinan_silver_tsv, arquivo_tsv), sep='\t', index=False)

C:\Users\Acer\AppData\Local\Temp\ipykernel_15084\3494526277.py:2: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(diretorio_sinan_bronze_csv, arquivo_csv), sep=',')


In [11]:
diretorio_dotenv = os.path.join(diretorio_base, ".env")

cursor, conn = toolkitEngineering.conectar_banco(diretorio_dotenv)
for arquivo_tsv in arquivos_tsv:
    
    caminho_arquivo = os.path.join(diretorio_sinan_silver_tsv, arquivo_tsv)
    toolkitEngineering.inserir_no_banco(cursor, conn, caminho_arquivo, 'sinan_dengue', delimiter='\t',
                                        diretorio_dotenv=diretorio_dotenv)
